## The dataset for this project can be found on
https://www.kaggle.com/techsash/waste-classification-data

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
keras = tf.keras

Using TensorFlow backend.


In [2]:
num_classes = 2
img_rows,img_cols = 224,224
batch_size = 32

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 0.3,
                                    zoom_range = 0.2,
                                    shear_range = 0.3,
                                    height_shift_range = 0.4,
                                    width_shift_range = 0.4,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    fill_mode = 'nearest')
validation_data_gen = ImageDataGenerator(rescale = 1./255)

In [4]:
train_data_dir = "/Users/dipit/Documents/MLDL/CNN/Waste Classification/datasets/train"
validation_data_dir = "/Users/dipit/Documents/MLDL/CNN/Waste Classification/datasets/test"

In [5]:
train_generator = train_data_gen.flow_from_directory(directory = train_data_dir,
                                                    target_size = (img_rows,img_cols),
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    class_mode = 'categorical')

Found 22564 images belonging to 2 classes.


In [6]:
validation_generator = validation_data_gen.flow_from_directory(directory = validation_data_dir,
                                                              target_size = (img_rows,img_cols),
                                                              batch_size = batch_size,
                                                              shuffle  = False,
                                                              class_mode = "categorical")

Found 2513 images belonging to 2 classes.


In [7]:
from keras.applications import MobileNet
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,BatchNormalization,Flatten,Activation
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import Adam

In [8]:
MobileNet = MobileNet(weights = 'imagenet',
                     include_top = False,
                     input_shape = (img_rows,img_cols,3))

for layer in MobileNet.layers:
    layer.trainable = False
    
    
for(i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " " + layer.__class__.__name__ , layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [9]:
from keras.layers import GlobalAveragePooling2D
def addTopModelMobileNet(bottom_model,num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation = 'relu')(top_model)
    top_model = Dense(1024,activation ='relu')(top_model)
    top_model = Dense(512,activation = 'relu')(top_model)
    top_model = Dense(num_classes,activation = 'softmax')(top_model)
    return top_model

In [10]:
FC_Head = addTopModelMobileNet(MobileNet,num_classes)
model = Model(MobileNet.input,outputs = FC_Head)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [11]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(lr = 0.001),
             metrics = ['accuracy'])

In [12]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                             factor = 0.2,
                             patience = 3,
                             verbose = 1,
                             min_delta = 0.0001)
callbacks = [reduce_lr]

In [13]:
number_train_samples = 22564
number_validation_samples = 2513
epochs = 5

In [14]:
hist = model.fit_generator(train_generator,
                          epochs = epochs,
                          steps_per_epoch = number_train_samples // batch_size,
                          validation_data = validation_generator,
                          validation_steps = number_validation_samples // batch_size,
                          callbacks = callbacks)

Epoch 1/5
705/705 [==============================] - 1238s 2s/step - loss: 0.2881 - accuracy: 0.8873 - val_loss: 0.3581 - val_accuracy: 0.9235
Epoch 2/5
705/705 [==============================] - 1212s 2s/step - loss: 0.2220 - accuracy: 0.9134 - val_loss: 0.5635 - val_accuracy: 0.8738
Epoch 3/5
705/705 [==============================] - 1215s 2s/step - loss: 0.2165 - accuracy: 0.9137 - val_loss: 0.8686 - val_accuracy: 0.8968
Epoch 4/5
705/705 [==============================] - 1215s 2s/step - loss: 0.1949 - accuracy: 0.9234 - val_loss: 0.2999 - val_accuracy: 0.9125

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/5
705/705 [==============================] - 1212s 2s/step - loss: 0.1758 - accuracy: 0.9301 - val_loss: 0.6528 - val_accuracy: 0.8960


In [15]:
scores = model.evaluate_generator(validation_generator,steps = number_validation_samples // batch_size+1,
                                 verbose = 1)
print("Test Loss:",scores[0])
print("Test Accuracy:",scores[1])

79/79 [==============================] - 127s 2s/step
Test Loss: 0.545407235622406
Test Accuracy: 0.8957421183586121


In [16]:
model.save("Waste_Classification.h5")

In [17]:
new_model = load_model("Waste_Classification.h5")

In [18]:
y_pred = new_model.predict_generator(validation_generator,steps = number_validation_samples // batch_size+1)

In [19]:
y_pred_label = np.argmax(y_pred,axis = 1)

In [20]:
from sklearn.metrics import classification_report,confusion_matrix
class_labels = validation_generator.class_indices
class_labels = {v : k for k,v in class_labels.items()}
classes = list(class_labels.values())
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes,y_pred_label))
print("Classification Rpeort")
print(classification_report(validation_generator.classes,y_pred_label,target_names = classes))

Confusion Matrix
[[1336   65]
 [ 197  915]]
Classification Rpeort
              precision    recall  f1-score   support

           O       0.87      0.95      0.91      1401
           R       0.93      0.82      0.87      1112

    accuracy                           0.90      2513
   macro avg       0.90      0.89      0.89      2513
weighted avg       0.90      0.90      0.89      2513



In [21]:
from keras.preprocessing import image
import os
import cv2
from os import listdir
from os.path import isfile ,join
import re

def draw_test(name,pred,im,true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im,160,0,0,300,cv2.BORDER_CONSTANT,value = BLACK)
    cv2.putText(expanded_image,"predicted-"+pred,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,225),2)
    cv2.putText(expanded_image,"True-"+true_label,(20,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow(name,expanded_image)
    
def getRandomImage(path,img_width,img_height):
    folders = list(filter(lambda x:os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + '/' + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path,target_size = (img_width,img_height)),final_path,path_class

img_width,img_height = 224,224

files = []
predictions = []
true_labels = []

for i in range(0,10):
    path = '/Users/dipit/Documents/MLDL/CNN/Waste Classification/datasets/test'
    img,final_path,true_label = getRandomImage(path,img_width,img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x*1./255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    classes = np.argmax(model.predict(images,batch_size = 10,verbose = 0),axis=1)
    predictions.append(classes)
    
for i in range(0,len(files)):
    image = cv2.imread((files[i]))
    draw_test("Prediction",class_labels[predictions[i][0]],image,true_labels[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()